In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import tensorflow as tf
import torch
import os
from datasets import Dataset
import time

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()  # Verifica antes de entrenar

Memoria GPU asignada: 0.00 MB
Memoria GPU reservada: 0.00 MB
Memoria libre: 4095.56 MB


In [4]:

def clear_gpu_memory():
    # Comando para restablecer la GPU usando nvidia-smi
    os.system('nvidia-smi --gpu-reset -i 0')

    # Limpiar la caché de memoria de CUDA
    torch.cuda.empty_cache()

    # También puedes intentar vaciar la caché de la memoria de TensorFlow, si lo estás usando
    try:
        import tensorflow as tf
        tf.keras.backend.clear_session()
        print("Sesión de TensorFlow limpiada.")
    except ImportError:
        pass

    # Verificación de memoria
    print("Verificando el estado de la memoria de la GPU después de limpiar...")
    time.sleep(2)  # Un pequeño retraso para asegurar que los cambios se apliquen
    os.system('nvidia-smi')

# Llamada a la función para limpiar la memoria de la GPU
clear_gpu_memory()


Sesión de TensorFlow limpiada.
Verificando el estado de la memoria de la GPU después de limpiar...


In [5]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()  # Carga variables del archivo .env
token = os.getenv("HF_TOKEN")
login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Asignar el token de padding al token de "fin de oración" (eos_token)
tokenizer.pad_token = tokenizer.eos_token

tokenizer.model_max_length = 1024  # Asegura que el tokenizador usa el límite correcto
model.to(device)  # Mover el modelo a la GPU si está disponible

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
"""
Carga de datos
"""
# Rutas de los archivos procesados
ruta_argumentos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\argumentos_procesados.csv"
ruta_dialogos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\dialogos_procesados.csv"
ruta_escritos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\escritos_procesados.csv"

# Cargar los datos
df_argumentos = pd.read_csv(ruta_argumentos)
df_dialogos = pd.read_csv(ruta_dialogos)
df_escritos = pd.read_csv(ruta_escritos)

# Combinar textos en un solo DataFrame
df = pd.concat([
    df_argumentos[['argumento']].rename(columns={'argumento': 'texto'}),
    df_dialogos[['dialogo']].rename(columns={'dialogo': 'texto'}),
    df_escritos[['contenido']].rename(columns={'contenido': 'texto'})
], ignore_index=True)

# Eliminar textos vacíos o nulos
df.dropna(subset=['texto'], inplace=True)

# Mostrar un ejemplo
print(df.sample(3))

                                                texto
83  La composición perfectamente estudiada del ,al...
45   Eutifrón ¿Qué novedad, Sócrates? ¿Abandonas t...
35   Sócrates Mira, Protarco, qué parte de la opin...


In [8]:
"""
Tokenización y preparación del Dataser
"""
# Cargar el modelo y el tokenizador GPT-2
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Configurar el token de padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 1024
model.to(device)

# Tokenización
def tokenize_function(examples):
    return tokenizer(examples['texto'], truncation=True, padding="max_length", max_length=1024)

# Convertir a Dataset de Hugging Face
from datasets import Dataset
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Dividir en entrenamiento y validación
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

Map: 100%|██████████| 99/99 [00:01<00:00, 66.60 examples/s]


In [9]:
"""
Configuracion de Entrenamiento
"""

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,               # Número de épocas de entrenamiento
    per_device_train_batch_size=4,    # Tamaño de lote para entrenamiento
    per_device_eval_batch_size=4,     # Tamaño de lote para validación
    evaluation_strategy="epoch",      # Evaluar al final de cada época
    save_strategy="epoch",            # Guardar modelo al final de cada época
    warmup_steps=500,                 # Calentamiento para el scheduler de LR
    weight_decay=0.01,                # Regularización L2
    logging_dir="./logs",             # Directorio para logs
    logging_steps=50,                 # Frecuencia de logging
    fp16=True if torch.cuda.is_available() else False  # Habilitar FP16 si hay GPU
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
"""
Entrenamiento del Modelo
"""
# Iniciar el entrenamiento
trainer.train()



  0%|          | 0/69 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
# Ruta para guardar el modelo entrenado
save_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\models"

# Guardar el modelo y el tokenizador
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Modelo y tokenizador guardados en: {save_path}")

In [14]:
"""
Evaluación rapida del modelo
"""
# Generar texto de prueba
input_text = "Sócrates dijo:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generar texto con el modelo entrenado
output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decodificar y mostrar el resultado
print(tokenizer.decode(output[0], skip_special_tokens=True))

''